In [1]:
import numpy as np
import visualization
import os
from gta_math import points_to_homo, ndc_to_view, construct_proj_matrix, view_to_world, construct_view_matrix, ndcs_to_pixels
from visualization import load_depth
from pointcloud_to_voxelmap import pointclouds_to_voxelmap
from PIL import Image
import pickle
from voxelmaps import camera_to_pointcloud, load_scene_db_data, get_main_image_name, scene_to_pointcloud, scene_to_voxelmap, NoMainImageException, scene_to_voxelmap_with_map, get_main_image
import voxelmaps

In [2]:
ini_file = "gta-postprocessing.ini"
visualization.multi_page = False
visualization.ini_file = ini_file

conn = visualization.get_connection()
cur = conn.cursor()


In [21]:
#run_id = 6
#run_id = 19
run_id = 3677

cur.execute("""SELECT snapshot_id, \
  ARRAY[st_x(camera_relative_rotation), st_y(camera_relative_rotation), st_z(camera_relative_rotation)] as camera_relative_rotation, \
  ARRAY[st_x(camera_pos), st_y(camera_pos), st_z(camera_pos)] as camera_pos \
  FROM snapshots \
  WHERE run_id = {} \
  AND camera_relative_rotation = st_makePoint(0, 0, 0) \
  ORDER BY timestamp ASC \
  """.format(run_id))

scenes = []
for row in cur:
    res = dict(row)
    res['camera_pos'] = np.array(res['camera_pos'])
    scenes.append(res)

print('There are {} scenes'.format(len(scenes)))

There are 959 scenes


In [10]:
positions = np.array([i['camera_pos'] for i in scenes])
diffs = np.diff(positions, axis=0)

In [11]:
diffs.shape

(22284, 3)

In [12]:
diffs_len = np.linalg.norm(diffs, axis=1)

In [13]:
diffs_len.shape

(22284,)

In [14]:
print('total driven length is {}'.format(diffs_len.sum()))

total driven length is 34340.62537517453


In [17]:
positions[0:6, :]

array([[  311.79238892, -1370.57763672,    31.82174301],
       [  311.7928772 , -1370.57617188,    31.82192612],
       [  311.79162598, -1370.54260254,    31.82613564],
       [  311.73892212, -1370.41589355,    31.83475685],
       [  311.60153198, -1370.24621582,    31.83952713],
       [  311.36730957, -1369.98449707,    31.84394455]])

In [15]:
diffs[0:5, :]

array([[  4.88281250e-04,   1.46484375e-03,   1.83105469e-04],
       [ -1.25122070e-03,   3.35693359e-02,   4.20951843e-03],
       [ -5.27038574e-02,   1.26708984e-01,   8.62121582e-03],
       [ -1.37390137e-01,   1.69677734e-01,   4.77027893e-03],
       [ -2.34222412e-01,   2.61718750e-01,   4.41741943e-03]])

In [16]:
np.linalg.norm(diffs[0:5, :], axis=1)

array([ 0.0015549 ,  0.03385537,  0.13750341,  0.21837889,  0.35124971])

In [20]:
np.sqrt((-1.37390137e-01)**2 + 1.69677734e-01**2 + 4.77027893e-03 **2)

0.21837888799360511